# Atak adwersarialny na filtr spamu przy użyciu TextAttack

Ten notatnik demonstruje, jak przeprowadzić atak zwodniczy na model wykrywania spamu przy użyciu biblioteki `textattack`. Użyjemy przepisu ataku `TextBugger` przeciwko filtrowi spamu opartemu na BERT-Tiny.

In [15]:
# Importowanie niezbędnych bibliotek
import textattack
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019, TextBuggerLi2018, DeepWordBugGao2018
from textattack import Attacker
from textattack.loggers import CSVLogger
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# textattack: Główna biblioteka do ataków adwersarzowych.
# transformers: Biblioteka Hugging Face do modeli NLP.
# HuggingFaceModelWrapper: Wrapper, który pozwala TextAttack korzystać z modeli Hugging Face.
# HuggingFaceDataset: Klasa do ładowania zbiorów danych z Hugging Face.
# TextFoolerJin2019, TextBuggerLi2018, DeepWordBugGao2018: Gotowe przepisy (recipes) ataków.
# Attacker: Klasa zarządzająca procesem ataku.
# CSVLogger: Do zapisywania wyników w pliku CSV.

## 1. Ładowanie modelu i tokenizera

Użyjemy `mrm8488/bert-tiny-finetuned-sms-spam-detection`, modelu BERT-Tiny dostrojonego na zbiorze danych SMS Spam. Ten model jest stosunkowo mały i dosyć podatny na ataki.

In [16]:
# Nazwa modelu do pobrania z Hugging Face Hub
model_name = "mrm8488/bert-tiny-finetuned-sms-spam-detection"

# Załadowanie modelu klasyfikacji sekwencji
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)
# Załadowanie tokenizera odpowiadającego modelowi
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# Opakowanie modelu i tokenizera w wrapper TextAttack, aby biblioteka mogła z nich korzystać
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

## 2. Załadowanie zbioru danych

Użyjemy zbioru danych `sms_spam` z biblioteki `datasets` Hugging Face. Uwaga: `sms_spam` zawiera tylko podział 'train', więc go użyjemy.

In [17]:
# Załaduj zbiór danych (sms_spam ma tylko podział 'train')
# Musimy również określić kolumny zbioru danych, ponieważ textattack nie rozpoznaje automatycznie 'sms' jako kolumny wejściowej
dataset = HuggingFaceDataset("sms_spam", split="train", shuffle=True, dataset_columns=(['sms'], 'label'))

textattack: Loading datasets dataset sms_spam, split train.


# 3.  Definicja Ataków Przeciwnych (Adversarial Attacks)

W tej analizie skupimy się na trzech metodach generowania przeciwnych przykładów tekstowych. Użyjemy **TextBugger** (`TextBuggerLi2018`) jako głównego ataku, a **TextFooler** i **DeepWordBug** jako alternatyw porównawczych.

---

## 3.1. Główny Atak: TextBugger (`TextBuggerLi2018`)

TextBugger jest **hybrydowym** atakiem typu **black-box**, zaprojektowanym do wprowadzania **minimalnych, zrozumiałych perturbacji** (zwanych "bugami") do tekstu. Jego celem jest spowodowanie błędnej klasyfikacji przez model docelowy przy jednoczesnym zachowaniu *utility* (zrozumiałości i użyteczności) dla ludzkiego oka.

###  Mechanizm działania

1.  **Identyfikacja Krytycznych Słów:** Atak najpierw wybiera słowa, które mają **największy wpływ** na pierwotną klasyfikację, mierząc spadek pewności modelu po ich usunięciu.
2.  **Generowanie Perturbacji (Bugów):** Dla każdego krytycznego słowa generowane są kandydatury zmian na dwóch poziomach:

| Poziom | Typ Perturbacji | Opis | Przykład (`love`) |
| :--- | :--- | :--- | :--- |
| **Słowo (Word)** | Podstawianie (Sub-W) | Zastąpienie słowa **synonimem** lub semantycznie bliskim sąsiadem (z wektorów embeddings lub WordNet). | `love` $\rightarrow$ `affection` |
| **Znak (Character)** | Wstawianie (Insert-C) | Wstawienie losowej litery lub spacji wewnątrz słowa. | `love` $\rightarrow$ `l ove` |
| **Znak (Character)** | Zamiana (Swap-C) | Zamiana sąsiednich znaków wewnątrz słowa. | `love` $\rightarrow$ `lvoe` |
| **Znak (Character)** | Podstawianie (Sub-C) | Zastąpienie znaku wizualnie podobnym (np. **homoglify**) lub sąsiadem z klawiatury. | `love` $\rightarrow$ `lovd` |

3.  **Optymalizacja:** Wybierany jest kandydat, który **maksymalnie zmienia predykcję** na błędną klasę, jednocześnie minimalizując liczbę użytych modyfikacji.

---

## 3.2. Alternatywne Ataki Porównawcze

Importujemy również `TextFooler` i `DeepWordBug` jako alternatywne strategie ataku, które różnią się metodologią perturbacji.

###  TextFooler (`TextFoolerJin2020`)

* **Charakterystyka:** Atak koncentrujący się wyłącznie na **podmianie słów** przy użyciu **synonimów**.
* **Kluczowa Cecha:** Po wybraniu krytycznych słów, synonimy są filtrowane przez **Universal Sentence Encoder (USE)**, aby zapewnić, że nowy tekst ma to samo znaczenie (zachowanie wysokiego podobieństwa semantycznego). Jest to kluczowe dla zachowania płynności i sensu zdania.

###  DeepWordBug (`DeepWordBugGao2018`)

* **Charakterystyka:** Atak skupiający się wyłącznie na **perturbacjach na poziomie znaków** (literówki), wykorzystujący bardziej zaawansowany mechanizm wyboru celu.
* **Kluczowa Cecha:** Jest to atak typu **white-box**, który wymaga dostępu do **gradientów** modelu. Używa techniki **Backward Differentiation** do obliczenia, które słowa są **najbardziej wrażliwe** na małe zmiany w swoich znakach, a następnie celuje w te znaki, aby maksymalnie zwiększyć stratę modelu.

In [18]:
# Wybierz przepis ataku (attack recipe)
attack_recipe = "TextBugger"

if attack_recipe == "TextBugger":
    # Zbuduj atak TextBugger dla naszego modelu
    attack = TextBuggerLi2018.build(model_wrapper)
elif attack_recipe == "TextFooler":
    # Zbuduj atak TextFooler
    attack = TextFoolerJin2019.build(model_wrapper)
elif attack_recipe == "DeepWordBug":
    # Zbuduj atak DeepWordBug
    attack = DeepWordBugGao2018.build(model_wrapper)

print(f"Używany przepis ataku: {attack_recipe}")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Używany przepis ataku: TextBugger


## 4. Uruchomienie ataku

Uruchomimy atak na 10 przykładach, aby zademonstrować jego skuteczność. Zapiszemy również wyniki do wizualizacji.

In [19]:
# Atakuj 10 próbek
# AttackArgs: argumenty dla ataku, takie jak liczba przykładów i logowanie do CSV
attack_args = textattack.AttackArgs(num_examples=10, log_to_csv="results_spam.csv")
# Attacker: obiekt wykonujący atak na zbiorze danych
attacker = Attacker(attack, dataset, attack_args)
# Uruchomienie ataku i pobranie wyników
results = attacker.attack_dataset()

textattack: Logging to CSV at path results_spam.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 








  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 5. Wizualizacja wyników

### Podświetlone różnice w tekście
Możemy wyświetlić oryginalny i zaburzony tekst z podświetlonymi różnicami.

In [ ]:
for result in results:
    # Wyświetl reprezentację HTML wyniku, pokazującą różnice
    display(HTML(result.__str__(color_method='html')))
    print("\n" + "="*50 + "\n")

### Statystyki podsumowujące

Spójrzmy na wskaźnik sukcesu i inne metryki.

In [1]:
# Oblicz statystyki
num_success = 0
num_failed = 0
num_skipped = 0

for result in results:
    # Sprawdź typ wyniku ataku
    if isinstance(result, textattack.attack_results.SuccessfulAttackResult):
        num_success += 1
    elif isinstance(result, textattack.attack_results.FailedAttackResult):
        num_failed += 1
    elif isinstance(result, textattack.attack_results.SkippedAttackResult):
        num_skipped += 1

print(f"Sukcesy: {num_success}")
print(f"Porażki: {num_failed}")
print(f"Pominięte: {num_skipped}")

# Rysowanie wykresu słupkowego
labels = ['Sukces', 'Porażka', 'Pominięte']
values = [num_success, num_failed, num_skipped]

plt.figure(figsize=(8, 6))
plt.bar(labels, values, color=['green', 'red', 'gray'])
plt.title('Podsumowanie wyników ataku')
plt.xlabel('Typ wyniku')
plt.ylabel('Liczba')
plt.show()

NameError: name 'results' is not defined

### Interpretacja wyników

Wykres słupkowy przedstawia podsumowanie skuteczności ataku adwersarzowego na wybranej próbce danych. Poniżej znajduje się wyjaśnienie poszczególnych kategorii:

- **Sukces (Success)**: Liczba przykładów, dla których atak zakończył się powodzeniem. Oznacza to, że algorytm zdołał zmodyfikować tekst wejściowy w taki sposób, że model zmienił swoją predykcję (np. z "spam" na "nie-spam"), zachowując jednocześnie semantyczne podobieństwo do oryginału.

- **Porażka (Failed)**: Liczba przykładów, dla których atak nie powiódł się. Algorytm nie był w stanie znaleźć takiej modyfikacji tekstu, która zmyliłaby model, przy zachowaniu zadanych ograniczeń (np. maksymalna liczba zmienionych słów, minimalne podobieństwo semantyczne).

- **Pominięte (Skipped)**: Liczba przykładów, które zostały pominięte w procesie ataku. Najczęściej dzieje się tak, gdy model błędnie sklasyfikował oryginalny tekst jeszcze przed atakiem. Ataki adwersarzowe zazwyczaj przeprowadza się tylko na poprawnie sklasyfikowanych przykładach, aby mierzyć skuteczność samego ataku, a nie błędy modelu.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from textattack.attack_results import SuccessfulAttackResult, FailedAttackResult, SkippedAttackResult

# 1. Obliczanie statystyk na podstawie rzeczywistych wyników ataku
total_attacks = len(results)
num_success = sum(1 for r in results if isinstance(r, SuccessfulAttackResult))
num_failed = sum(1 for r in results if isinstance(r, FailedAttackResult))
num_skipped = sum(1 for r in results if isinstance(r, SkippedAttackResult))

# Obliczanie dokładności (Accuracy) i skuteczności (Success Rate)
# Original Accuracy = (Succeeded + Failed) / Total
original_accuracy = ((num_success + num_failed) / total_attacks) * 100 if total_attacks > 0 else 0
# Accuracy under Attack = Failed / Total
accuracy_under_attack = (num_failed / total_attacks) * 100 if total_attacks > 0 else 0
# Attack Success Rate = Succeeded / (Succeeded + Failed)
attack_success_rate = (num_success / (num_success + num_failed)) * 100 if (num_success + num_failed) > 0 else 0

# Obliczanie średniego stopnia perturbacji (zmiany słów)
perturbation_percentages = []
for r in results:
    if isinstance(r, SuccessfulAttackResult):
        orig_words = len(r.original_result.attacked_text.words)
        pert_words = r.num_words_changed
        perturbation_percentages.append((pert_words / orig_words) * 100 if orig_words > 0 else 0)

avg_pert = sum(perturbation_percentages) / len(perturbation_percentages) if perturbation_percentages else 0

# 2. Tworzenie syntetycznej tabeli wyników
summary_data = {
    "Metryka": [
        "Liczba analizowanych próbek",
        "Udane ataki (Succeeded)",
        "Nieudane ataki (Failed)",
        "Pominięte (Skipped - model już się mylił)",
        "Dokładność przed atakiem",
        "Dokładność po ataku",
        "Skuteczność ataku (Success Rate)",
        "Średni % zmienionych słów"
    ],
    "Wartość": [
        total_attacks,
        num_success,
        num_failed,
        num_skipped,
        f"{original_accuracy:.1f}%",
        f"{accuracy_under_attack:.1f}%",
        f"{attack_success_rate:.1f}%",
        f"{avg_pert:.1f}%"
    ]
}

df_summary = pd.DataFrame(summary_data)
print("### ZESTAWIENIE WYNIKÓW ATAKU ADWERSARIALNEGO ###")
display(df_summary)

# 3. Wizualizacja wpływu ataku na skuteczność filtra
plt.figure(figsize=(10, 6))
labels = ['Dokładność Oryginalna', 'Dokładność pod Atakiem']
values = [original_accuracy, accuracy_under_attack]
colors = ['#2ecc71', '#e74c3c']

bars = plt.bar(labels, values, color=colors, width=0.6)
plt.ylabel('Dokładność (%)')
plt.title(f'Wpływ ataku {attack_recipe} na filtr spamu (BERT-Tiny)')
plt.ylim(0, 100)

# Dodanie etykiet procentowych nad słupkami
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 2, f'{yval:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()

# 4. Analiza i wnioski
print("\n### ANALIZA I WNIOSKI ###")
analysis = f"""
1. ILE (Skala zjawiska):
   - Atakujący zdołał oszukać model w {num_success} na {num_success + num_failed} przypadkach, w których model początkowo działał poprawnie.
   - Skuteczność ataku (Success Rate) wyniosła {attack_success_rate:.1f}%, co drastycznie obniżyło realną wartość filtra spamu.

2. JAK (Metoda manipulacji):
   - Metoda {attack_recipe} (TextBugger) wykorzystała hybrydowe podejście:
     - Na poziomie słów: zamiana na synonimy (np. 'urgent' -> 'immediate').
     - Na poziomie znaków: wprowadzanie "bugów" (literówki, zamiany znaków na wizualnie podobne, np. 'o' -> '0').
   - Średnio wystarczyło zmienić zaledwie {avg_pert:.1f}% tekstu, aby wiadomość sklasyfikowana jako SPAM została uznana za bezpieczną (HAM).

3. JAKI WPŁYW (Konsekwencje):
   - Model BERT-Tiny, mimo wysokiej wydajności, wykazuje dużą wrażliwość na drobne perturbacje znakowe.
   - Dla użytkownika końcowego wiadomość pozostaje w pełni czytelna (np. "C0ngratulations!"), ale dla algorytmu
     wektor wejściowy zmienia się na tyle, że predykcja zostaje całkowicie odwrócona.
   - Wniosek: Filtry spamu oparte na prostych transformerach wymagają dodatkowego wzmocnienia (np. Adversarial Training)
     lub filtrów normalizujących tekst przed klasyfikacją.
"""
print(analysis)